In [5]:
import pandas as pd

df = pd.read_csv('BC SPCA _ Adopatable List_Revision_Completed.csv')

In [ ]:
df.printSchema()

root
 |-- Pet: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Breed: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Province: string (nullable = true)
 |-- First_Letter: string (nullable = true)

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Assuming you have a CSV file "data.csv"
df = spark.read.csv("BC SPCA _ Adopatable List_Revision_Completed.csv", header=True, inferSchema=True)

df.select(['Pet','Name','Age','Sex']).show(5)

+---+---------+---------+------+
|Pet|     Name|      Age|   Sex|
+---+---------+---------+------+
|Dog|Big Daddy|10 months|  Male|
|Dog|     Sven|   1 year|  Male|
|Dog|   Stormy|  5 years|  Male|
|Dog|    Sasha|  4 years|Female|
|Dog|  Lillian|  2 years|Female|
+---+---------+---------+------+
only showing top 5 rows



In [18]:
df.head()

,Pet,Name,Sex,Age,Breed,Center Location,Province
0,Dog,Big Daddy,Male,10 months,Pit bull,Vancouver,B.C.
1,Dog,Sven,Male,1 year old,Mastiff,Vancouver,B.C.
2,Dog,Stormy,Male,5 years old,Domestic Medium Hair,Vancouver,B.C.
3,Dog,Sasha,Female,4 years old,Doberman,Vancouver,B.C.
4,Dog,Lillian,Female,2 years old,Sheepdog,Vancouver,B.C.


In [31]:
vancouver_data = df[df['Center Location'] == 'Vancouver']
print("The following pets are in Vancouver:")
for pet in vancouver_data['Name']:
    print(f"- {pet}")

The following pets are in Vancouver:
- Big Daddy
- Sven
- Stormy
- Sasha
- Lillian
- Lillian
- Stormy
- Berry
- Bubba
- Bert
- Belle
- Cashmere
- Milly
- Luna
- Hudson


In [14]:
richmond_data = df[df['Location'] == 'Richmond']
print("The following pets are in Richmond:")
for pet in richmond_data.select('Name').rdd.flatMap(lambda x: x).collect():
    print(f"- {pet}")

The following pets are in Richmond:


- Lucy
- Pluto
- Grace
- Stanley
- Dusty
- Mick
- Babie
- Kasey
- Buttons 
- Rocky
- Pixel 
- Dr. Patterson


In [49]:
# Replace 'AgeInYears' with the actual column name in your CSV file
oldest_pet = df[df['Age'] == df['Age'].max()]

# Print the oldest pet
print(oldest_pet)


    Pet    Name     Sex      Age                 Breed Center Location  \
16  Dog   Dusty    Male  9 years       German Shepherd        Richmond   
27  Dog   Suzie  Female  9 years             Chihuahua        Victoria   
31  Cat  Stormy    Male  9 years  Domestic Medium Hair       Vancouver   

   Province  AgeInYears     AgeGroup  
16    B.C.          NaN  Over 1 year  
27     B.C.         NaN  Over 1 year  
31    B.C.          NaN  Over 1 year  


In [50]:
youngest_pet = df[df['Age'] == df['Age'].min()]

# Print the youngest pet
print(youngest_pet)

           Pet           Name     Sex      Age Breed Center Location Province  \
85  Guinea pig  Pumpkin Spice  Female  1 month   NaN      Abbotsford    B.C.    

    AgeInYears          AgeGroup  
85    0.083333  Less than 1 year  


In [53]:
# Group by 'Gender' and 'Pet', then count
pet_counts = df.groupby(['Sex', 'Pet']).size()

# Print the result
print(pet_counts)

Sex     Pet       
Female  Cat           20
        Dog           13
        Guinea pig     5
        Rabbit        18
Male    Cat           10
        Dog           17
        Guinea pig     9
        Rabbit         7
dtype: int64


In [52]:
# Replace 'PetType' and 'Sex' with the actual column names in your CSV file
grouped = df.groupby(['Pet', 'Sex']).size()
percentages = grouped.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

# Print the percentages
print(percentages)


Pet         Pet         Sex   
Cat         Cat         Female    66.666667
                        Male      33.333333
Dog         Dog         Female    43.333333
                        Male      56.666667
Guinea pig  Guinea pig  Female    35.714286
                        Male      64.285714
Rabbit      Rabbit      Female    72.000000
                        Male      28.000000
dtype: float64


In [62]:
# Replace 'PetName' with the actual column name in your CSV file
df['StartingLetter'] = df['Name'].str[0]
letter_counts = df['StartingLetter'].value_counts()
most_common_starting_letter = letter_counts.idxmax()
number_of_most_common_starting_letter = letter_counts.max()

print(f"The most common starting letter for pet names is: {most_common_starting_letter}")
print(f"The number of pet names that start with this letter is: {number_of_most_common_starting_letter}")



The most common starting letter for pet names is: S
The number of pet names that start with this letter is: 18


In [65]:

# Replace 'Age' with the actual column name in your CSV file
def convert_age(age_str):
    # Split the age string into parts
    parts = age_str.split()

    # Convert the age to years
    if 'month' in parts:
        return int(parts[0]) / 12
    elif 'year' in parts:
        return int(parts[0]) + (int(parts[2]) / 12 if len(parts) > 2 else 0)
    else:
        return None

df['AgeInYears'] = df['Age'].apply(convert_age)

# Count the total numbers of pets which are from 1 year to 3 years old
count = df[(df['AgeInYears'] >= 1) & (df['AgeInYears'] <= 3)].shape[0]

print(f"The total number of pets which are from 1 year to 3 years old is: {count}")


The total number of pets which are from 1 year to 3 years old is: 20


In [75]:
import pandas as pd

# Function to convert age to months
def convert_age_to_months(age):
    age_value, age_unit = age.split()
    age_value = int(age_value)
    if 'month' in age_unit:
        return age_value
    elif 'year' in age_unit:
        return age_value * 12

# Load the data from CSV file
df = pd.read_csv('BC SPCA _ Adopatable List_Revision_Completed.csv')

# Convert the 'Age' column to months
df['Age_in_months'] = df['Age'].apply(convert_age_to_months)

# Filter the data for pets older than 3 years (36 months)
older_pets = df[df['Age_in_months'] > 36]

# Print the total number of pets older than 3 years
print(f'Total number of pets older than 3 years: {len(older_pets)}')


Total number of pets older than 3 years: 23


In [76]:
# Filter the data for pets younger than 1 year (12 months)
younger_pets = df[df['Age_in_months'] < 12]

# Print the total number of pets younger than 1 year
print(f'Total number of pets younger than 1 year: {len(younger_pets)}')


Total number of pets younger than 1 year: 24


In [82]:

# Convert the 'Age' column to months
df['Age_in_months'] = df['Age'].apply(convert_age_to_months)

# Total number of pets
total_pets = len(df)

# Number of pets younger than 1 year
younger_pets = len(df[df['Age_in_months'] < 12])

# Calculate the percentage
percentage = (younger_pets / total_pets) * 100

# Print the results
print(f'Total number of pets: {total_pets}')
print(f'Number of pets younger than 1 year: {younger_pets} ({percentage:.2f}%)')


Total number of pets: 99
Number of pets younger than 1 year: 24 (24.24%)


In [91]:
import pandas as pd
import numpy as np

# Define the age groups
bins = [0, 12, 24, 36, 48, 60, 72, np.inf]
labels = ['<1 year', '1-2 years', '2-3 years', '3-4 years', '5-6 years', '7-8 years','9+ years']

# Create a new column 'Age_group' for age groups
df['Age_group'] = pd.cut(df['Age_in_months'], bins=bins, labels=labels)

# Filter the data for dogs and cats
dogs_and_cats = df[df['Pet'].isin(['Dog', 'Cat'])]

# Get the total number in each age group for dogs and cats
grouped_data = dogs_and_cats.groupby(['Pet', 'Age_group']).size()

# Print the results
print(grouped_data)



Pet  Age_group
Cat  <1 year      10
     1-2 years     3
     2-3 years     5
     3-4 years     3
     5-6 years     1
     7-8 years     2
     9+ years      6
Dog  <1 year       9
     1-2 years     8
     2-3 years     3
     3-4 years     2
     5-6 years     2
     7-8 years     3
     9+ years      3
dtype: int64


/var/folders/kk/4by1cf017_b103mywlfcgm580000gp/T/ipykernel_7712/509509536.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = dogs_and_cats.groupby(['Pet', 'Age_group']).size()


In [93]:
import pandas as pd
import numpy as np

# Define the age groups
bins = [0, 12, 24, 36, 48, 60, 72, np.inf]
labels = ['<1 year', '1-2 years', '2-3 years', '3-4 years', '5-6 years', '7-8 years','9+ years']

# Create a new column 'Age_group' for age groups
df['Age_group'] = pd.cut(df['Age_in_months'], bins=bins, labels=labels)

# Filter the data for rabbits
rabbits = df[df['Pet'] == 'Rabbit']

# Get the total number in each age group for rabbits
grouped_data = rabbits.groupby(['Age_group']).size()

# Print the results
print(grouped_data)

Age_group
<1 year      18
1-2 years     4
2-3 years     2
3-4 years     1
5-6 years     0
7-8 years     0
9+ years      0
dtype: int64


/var/folders/kk/4by1cf017_b103mywlfcgm580000gp/T/ipykernel_7712/891359549.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = rabbits.groupby(['Age_group']).size()


In [6]:
from pyspark.sql.functions import col, length
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Assuming 'Name' is the column with pet names and 'Sex' is the column with pet sex
df = spark.read.csv('BC SPCA _ Adopatable List_Revision_Completed.csv', header=True)

# Create a new column 'First_Letter' with the first letter of each pet name
df = df.withColumn('First_Letter', df['Name'].substr(1, 1))

# Filter rows where 'First_Letter' is not null and 'Name' length is greater than 0
df = df.filter((col('First_Letter').isNotNull()) & (length(col('Name')) > 0))

# Group by 'Sex' and 'First_Letter', and count
result = df.groupBy('Sex', 'First_Letter').count()

# Show the result
result.show()


24/01/04 20:19:48 WARN Utils: Your hostname, URYs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.82 instead (on interface en0)
24/01/04 20:19:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 20:19:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------+------------+-----+
|   Sex|First_Letter|count|
+------+------------+-----+
|Female|           M|    5|
|  Male|           P|    2|
|Female|           D|    3|
|  Male|           H|    3|
|  Male|           L|    2|
|Female|           K|    2|
|Female|           G|    2|
|Female|           N|    1|
|  Male|           B|    9|
|Female|           P|    4|
|Female|           O|    1|
|  Male|           C|    1|
|Female|           Z|    1|
|  Male|           D|    3|
|  Male|           F|    2|
|Female|           J|    1|
|Female|           A|    3|
|Female|           R|    2|
|Female|           B|    4|
|Female|           E|    1|
+------+------------+-----+
only showing top 20 rows



24/01/04 20:20:09 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
# Group by 'Sex' and count
result = df.groupBy('Sex').count()

# Show the result
result.show()


+------+-----+
|   Sex|count|
+------+-----+
|Female|   56|
|  Male|   43|
+------+-----+



In [9]:
from pyspark.sql.functions import col, length

# Assuming 'Name' is the column with pet names and 'Sex' is the column with pet sex
df = spark.read.csv('BC SPCA _ Adopatable List_Revision_Completed.csv', header=True)

# Create a new column 'First_Letter' with the first letter of each pet name
df = df.withColumn('First_Letter', df['Name'].substr(1, 1))

# Filter rows where 'First_Letter' is not null and 'Name' length is greater than 0
df = df.filter((col('First_Letter').isNotNull()) & (length(col('Name')) > 0))

# Group by 'Sex' and 'First_Letter', and count
result = df.groupBy('Sex', 'First_Letter').count()

# Sort the result by 'Sex' and 'First_Letter'
result = result.orderBy('Sex', 'First_Letter')

# Show the result
result.show(99)


+------+------------+-----+
|   Sex|First_Letter|count|
+------+------------+-----+
|Female|           A|    3|
|Female|           B|    4|
|Female|           C|    3|
|Female|           D|    3|
|Female|           E|    1|
|Female|           G|    2|
|Female|           H|    4|
|Female|           I|    1|
|Female|           J|    1|
|Female|           K|    2|
|Female|           L|    6|
|Female|           M|    5|
|Female|           N|    1|
|Female|           O|    1|
|Female|           P|    4|
|Female|           R|    2|
|Female|           S|    9|
|Female|           W|    2|
|Female|           Y|    1|
|Female|           Z|    1|
|  Male|           A|    3|
|  Male|           B|    9|
|  Male|           C|    1|
|  Male|           D|    3|
|  Male|           F|    2|
|  Male|           G|    1|
|  Male|           H|    3|
|  Male|           K|    2|
|  Male|           L|    2|
|  Male|           M|    1|
|  Male|           N|    1|
|  Male|           P|    2|
|  Male|           S

In [19]:
from pyspark.sql import functions as F

# Assuming 'Type' is the column in your DataFrame that contains the type of pet
types = ['Dog', 'Cat', 'Rabbit']

for pet_type in types:
    print(f"Most common breed among {pet_type}s:")
    
    # Filter DataFrame for the current type of pet
    df_type = df[df['Pet'] == pet_type]
    
    # Group by 'Breed' and count
    breed_counts = df_type.groupBy('Breed').count()
    
    # Sort in descending order of count
    sorted_breed_counts = breed_counts.sort(F.desc('count'))
    
    # Show the breed with the highest count
    sorted_breed_counts.show(3)


Most common breed among Dogs:
+------------------+-----+
|             Breed|count|
+------------------+-----+
|Labrador Retriever|    3|
|  Pit Bull Terrier|    3|
|   German Shepherd|    3|
+------------------+-----+
only showing top 3 rows

Most common breed among Cats:
+--------------------+-----+
|               Breed|count|
+--------------------+-----+
| Domestic Short Hair|   25|
|Domestic Medium Hair|    3|
|  Domestic Long Hair|    1|
+--------------------+-----+
only showing top 3 rows

Most common breed among Rabbits:
+--------------------+-----+
|               Breed|count|
+--------------------+-----+
|   Californian Dwarf|    8|
|            American|    3|
|Lionhead/Flemish ...|    2|
+--------------------+-----+
only showing top 3 rows



24/01/05 16:34:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 915674 ms exceeds timeout 120000 ms
24/01/05 16:34:40 WARN SparkContext: Killing executors is not supported by current scheduler.
24/01/05 16:34:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [13]:
df.printSchema()


root
 |-- Pet: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Breed: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Province: string (nullable = true)
 |-- First_Letter: string (nullable = true)

